# Jupyter Notebook: [The Hidden Treasure of _delta_logֲֲֲֲ.](https://blog.devgenius.io/the-hidden-treasure-of-delta-log-482684efb36b)

## Setup and Imports.

In [ ]:
import time
import polars as pl
from deltalake import DeltaTable

## Data Preparation and DataFrame Creation.

In [ ]:
table_path = './deltalake/test_table'
table_part_path = './deltalake/test_table_part'
delta_write_options = {"partition_by": ["client"]}

test_data_1 = {
    'id' : ['xxx1', 'xxx2', 'xxx3', 'xxx4'],
    'client' : ['0001', '0001', '0002', '0002'],
    'item' : ['paper', 'ball','pen','pen'],
    'price': [5, 1, 2, 2]
}

test_data_2 = {
    'id' : ['xxx1', 'xxx2', 'xxx3', 'xxx4'],
    'client' : ['0001', '0001', '0002', '0003'],
    'item' : ['paper', 'ball','pen','pen'],
    'price': [5, 1, 2, 2]
}

test_data_3 = {
    'id' : ['xxx4'],
    'client' : ['0003'],
    'item' : ['paper'],
    'price': [5]
}

test_data_4 = {
    'id' : ['xxx1',],
    'client' : ['0001'],
    'item' : ['paper'],
    'price': [6]
}

df_1 = pl.DataFrame(test_data_1)
df_2 = pl.DataFrame(test_data_2)
df_3 = pl.DataFrame(test_data_3)
df_4 = pl.DataFrame(test_data_4)

## Writes DataFrames to Delta Lake.

In [ ]:
time_sleep_seconds:int = 60

df_1.write_delta(
    target=table_path,
    mode="append",
)

df_1.write_delta(
    target=table_part_path,
    delta_write_options=delta_write_options,
    mode="append",
)

time.sleep(time_sleep_seconds)

df_2.write_delta(
    target=table_path,
    mode="overwrite",
)

df_2.write_delta(
    target=table_part_path,
    mode="overwrite",
)

time.sleep(time_sleep_seconds)

df_3.write_delta(
    target=table_path,
    mode='merge',
    delta_merge_options={
        'predicate': 'src.id = trg.id',
        'source_alias': 'src',
        'target_alias': 'trg',
    },
) \
.when_matched_update_all() \
.execute()

df_3.write_delta(
    target=table_part_path,
    mode='merge',
    delta_merge_options={
        'predicate': 'src.id = trg.id',
        'source_alias': 'src',
        'target_alias': 'trg',
    },
) \
.when_matched_update_all() \
.execute()

time.sleep(time_sleep_seconds)

df_4.write_delta(
    target=table_path,
    mode='merge',
    delta_merge_options={
        'predicate': 'src.id = trg.id',
        'source_alias': 'src',
        'target_alias': 'trg',
    },
) \
.when_matched_update_all() \
.execute()

df_4.write_delta(
    target=table_part_path,
    mode='merge',
    delta_merge_options={
        'predicate': 'src.id = trg.id',
        'source_alias': 'src',
        'target_alias': 'trg',
    },
) \
.when_matched_update_all() \
.execute()

## Inspects a _delta_log.

In [ ]:
!cat ./deltalake/test_table/_delta_log/00000000000000000000.json | jq .

## Retrieves a dataframe with all current add actions without partitions.

In [ ]:
dt: DeltaTable = DeltaTable(table_path)
actions = dt.get_add_actions() # pyarrow.lib.RecordBatch
df: pl.DataFrame = pl.from_arrow(actions)
df

## Retrieves a dataframe with all current add actions with partitions.

In [ ]:
dt_part: DeltaTable = DeltaTable(table_part_path)
df_part: pl.DataFrame = pl.from_arrow(dt_part.get_add_actions())
df_part

## Gets all add actions for all time from a delta table.

In [ ]:
dt_version: int = 0 if dt.version() == 0 else dt.version() + 1
df_vers: pl.DataFrame = pl.concat(
    [
        pl.from_arrow(
            DeltaTable(table_path, version=v).get_add_actions(flatten=True)
        )
        for v in range(0, dt_version)
    ]
).unique().sort("modification_time")
df_vers